In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import math
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn

In [ ]:
import os
print(os.getcwd())

In [ ]:
df = pd.read_csv("../../models/training_history.csv")

In [ ]:
df.head()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(14, 5))

# Plot Loss
axs[0].plot(df['epoch'], df['train_loss'], label='Train Loss')
axs[0].plot(df['epoch'], df['test_loss'], label='Test Loss')
axs[0].set_title('Loss over Epochs')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Loss')
axs[0].legend()
axs[0].grid(True)

# Plot MAE
axs[1].plot(df['epoch'], df['train_mae'], label='Train MAE')
axs[1].plot(df['epoch'], df['test_mae'], label='Test MAE')
axs[1].set_title('MAE over Epochs')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Mean Absolute Error')
axs[1].legend()
axs[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:


# Define the model class again (must match your original)
import torch.nn as nn
class VoltageRegressor(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 360),
            nn.ReLU(),
            nn.Linear(360, 90),
            nn.ReLU(),
            nn.Linear(90, 30),
            nn.ReLU(),
            nn.Linear(30, 1)
        )
    def forward(self, x):
        return self.model(x)

# Load data
df = pd.read_csv('../../data/DS4.csv')
target_col = 'delta_V'
exclude_cols = [
    'Unnamed: 0',
    'framework',
    'charged_id',
    'charged_formula',
    'discharged_id',
    'discharged_formula',
    'active_metals',
    'composition',
    target_col
]
df = df.replace([np.inf, -np.inf], np.nan).dropna().reset_index(drop=True)

X = df.drop(columns=exclude_cols).values.astype(np.float32)
y = df[target_col].values.astype(np.float32).reshape(-1, 1)

# Split train/test same way as training (or load your test split if saved)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Scale features with StandardScaler (fit on train only)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Prepare test tensor
X_test_t = torch.tensor(X_test)

# Load model
input_dim = X_train.shape[1]
model = VoltageRegressor(input_dim)
model.load_state_dict(torch.load('../../models/DNN1.pth'))
model.eval()

# Predict on test set
with torch.no_grad():
    preds = model(X_test_t).numpy()

# Compute residuals
residuals = y_test - preds

# Plot residuals vs predicted
plt.figure(figsize=(8,6))
plt.scatter(preds, residuals, alpha=0.5, s=10)
plt.axhline(0, color='red', linestyle='--')
plt.xlabel('Predicted Values')
plt.ylabel('Residuals (Actual - Predicted)')
plt.title('Residuals vs Predicted Values')
plt.show()

# Plot histogram of residuals
plt.figure(figsize=(8,6))
plt.hist(residuals, bins=50, alpha=0.7)
plt.xlabel('Residual')
plt.ylabel('Frequency')
plt.title('Histogram of Residuals')
plt.show()


In [ ]:
mse = mean_squared_error(y_test, preds)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, preds)
r2 = r2_score(y_test, preds)

print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"R^2: {r2:.4f}")